In [30]:
import numpy as np
R = 81
eta = 3
s_max = int(np.log(R) / np.log(eta))
B = (s_max + 1)*R
b = 0
for s in range(0, s_max+1):
    n = int(np.ceil((B/R)*(eta**s)/(s+1)))
    r = R*eta**(-s)
    # T = np.arange(n+1)
    for i in range(s+1):
        n_i = int(n*eta**(-i))
        r_i = int(r*(eta**i))
        r_im = int(r*(eta**(i-1))) if i >= 0 else 0
        rd = int(5*r_i) # Without eval at round 5, will use eval at round 0.
        b += n_i*(r_i - r_im)*5
b

6475

In [ ]:
runs = set([k[0] for k in ed.keys()])
hp_data = {}
for run in runs:
    if (run, 405, 0.0) not in ed:
    # if (run, 405) not in ed:
        continue
    hps = ['lr', 'momentum', 'beta1', 'beta2']
    # hps = ['lr', 'momentum']
    vals = []
    # sub_run = run.split('-beta1')[0]
    # sub_run = run.split('-CLIENT')[0]
    sub_run = run.split('-betas')
    lr = sub_run[0].split('opt_adam-lr_')[1]
    hp_data[float(lr)] = ed[run, 405, 0.0]
    # seed = int(sub_run.split('-')[1])
    # if seed >= 16:
    #     continue
    # for i, hp in enumerate(hps):
    #     val, sub_run = sub_run.split('-' + hp + '_')
    #     if i != 0:
    #         vals.append(float(val[:7]))
    # vals.append(float(sub_run[:7]))
    # hp_data[seed, vals[0], vals[2], vals[3]] = ed[run, 405]

In [ ]:
grid_evals = {k: hp_data[k]*sizes[d] for k in hp_data}

In [ ]:
from collections import defaultdict
# rs_results = defaultdict(lambda: [])

# d='femnist'
# NC = 360
d = 'cifar10'
NC = 100
# LR = 2.0**np.arange(-5, 1)
LR = 10.0**np.arange(-6, 0)
# N = 256
N = 16
# N = 64

# sgd
# lr_runs = {
#     'fixed': [k for k in hp_data if k[1] == lr and k[2] == m],
#     'mu': [k for k in hp_data if k[1] == lr],
#     'all': [k for k in hp_data],
# }
# adam
lr = 0.001
b1 = 0.9
b2 = 0.999
grids = {
    '$\eta \in [10^{-4.5}, 10^{-4}]$': [k for k in hp_data if k > 10**(-4.5) and k < 10**(-4)],
    '$\eta \in [10^{-5}, 10^{-3.5}]$': [k for k in hp_data if k > 10**(-5) and k < 10**(-3.5)],
    '$\eta \in [10^{-5.5}, 10^{-3}]$': [k for k in hp_data if k > 10**(-5.5) and k < 10**(-3)],
    '$\eta \in [10^{-6}, 10^{-2.5}]$': [k for k in hp_data if k > 10**(-6) and k < 10**(-2.5)],
    # 'Default': [k for k in hp_data if k[1] == lr and k[2] == b1 and k[3] == b2],
    # 'Tune only $\\eta$': [k for k in hp_data if k[2] == b1 and k[3] == b2],
    # 'Tune only $\\beta_1$': [k for k in hp_data if k[1] == lr and k[3] == b2],
    # 'Tune only $\\beta_2$': [k for k in hp_data if k[1] == lr and k[2] == b1],
    # 'Tune $\\eta$ and $\\beta_1$': [k for k in hp_data if k[3] == b2],
    # 'Tune $\\eta$ and $\\beta_2$': [k for k in hp_data if k[2] == b1],
    # 'Tune $\\beta_1$ and $\\beta_2$': [k for k in hp_data if k[1] == lr],
    # 'Tune $\\eta, \\beta_1, \\beta_2$': [k for k in hp_data]
}
for nc in [1,NC]:
    for label, lr_runs in grids.items():
        lr_results = []
        if nc == NC:
            evals = []
            for k in lr_runs:
                eval = grid_evals[k].sum() / sizes[d].sum()
                evals.append(eval)
            lr_evals = np.array(evals)
        
        for trial in tqdm(range(8000)):
            rs_runs = np.random.choice(len(lr_runs), N, replace=True)

            if nc < NC:      
                rs_evals = []
                for i in rs_runs:
                    k = lr_runs[i]
                    eval_idx = np.random.choice(NC, nc, replace=False)
                    eval = grid_evals[k][eval_idx].sum() / sizes[d][eval_idx].sum()
                    eps = 1
                    noise = np.random.laplace(scale=N/(nc*eps)) if eps is not None else 0
                    eval = np.clip(eval+noise, 0, 1)
                    rs_evals.append(eval)
                # rs_evals = np.array(evals)
                min_idx = np.argmin(rs_evals)
            else:
                min_idx = np.argmin([lr_evals[i] for i in rs_runs])

            true_eval = grid_evals[lr_runs[rs_runs[min_idx]]].sum() / sizes[d].sum()
            lr_results.append(true_eval)
        rs_results[label, nc] = np.array(lr_results)

In [ ]:
# ts = {
#     'Default': [k for k in hp_data if k[1] == lr and k[2] == b1 and k[3] == b2],
#     'Tune only $\\eta$': [k for k in hp_data if k[2] == b1 and k[3] == b2],
#     'Tune only $\\beta_1$': [k for k in hp_data if k[1] == lr and k[3] == b2],
#     'Tune only $\\beta_2$': [k for k in hp_data if k[1] == lr and k[2] == b1],
#     'Tune $\\eta$ and $\\beta_1$': [k for k in hp_data if k[3] == b2],
#     'Tune $\\eta$ and $\\beta_2$': [k for k in hp_data if k[2] == b1],
#     'Tune $\\beta_1$ and $\\beta_2$': [k for k in hp_data if k[1] == lr],
#     'Tune $\\eta, \\beta_1, \\beta_2$': [k for k in hp_data],
ts = grids

In [ ]:
import matplotlib.pyplot as plt
c = ['red', 'blue', 'green', 'orange', 'white', 'cyan' ,'lime', 'pink']
# c = ['red', 'blue', 'green', 'orange']
# NCS = [1,3,9,27,81,NC]
# for j, t in enumerate(grids):
#     m = np.array([np.median(rs_results[t, nc]) for nc in NCS])
#     q1 = np.array([np.quantile(rs_results[t, nc], 0.25) for nc in NCS])
#     q2 = np.array([np.quantile(rs_results[t, nc], 0.75) for nc in NCS])
#     plt.plot(NCS, m,label=t)
#     plt.fill_between(NCS, q1, q2,alpha=0.2)
for j, t in enumerate(ts):
    evals = [grid_evals[k].sum() / sizes[d].sum() for k in grids[t]]
    q1 = min(evals)
    q2 = max(evals)
    plt.errorbar([j], [np.median(evals)], yerr=[[np.median(evals) - q1],[q2 - np.median(evals)]], marker='o', color='skyblue', capsize=3)

    nc = NC
    m = np.median(rs_results[t, nc])
    # q1 = np.quantile(rs_results[t, nc], 0.25)
    # q2 = np.quantile(rs_results[t, nc], 0.75)
    # q1 = np.min(rs_results[t, nc])
    # q2 = np.max(rs_results[t, nc])
    plt.bar(
        [j], [m], width=1, color=c[j], 
        capsize=3, edgecolor='k', linewidth=2,
        label=f"{NC} Clients" if j == 0 else None,alpha=0.6)
    nc = 1
    m1 = np.median(rs_results[t, nc])
    plt.bar(
        [j], [m1 - m], width=1, color=c[j], 
        bottom = [m],
        capsize=3, edgecolor='k', linewidth=2,
        alpha=0.6, hatch='\\\\',
        label="1 Client, $\epsilon=1$" if j == 0 else None)
plt.legend(ncol=2, loc='lower center')
plt.xticks([i for i in range(len(ts))], ts.keys(), rotation=45, ha='right')
# plt.xlabel('Num. of clients')
# plt.ylim(0.13, 0.18)
# plt.ylim(0.3, 0.4)
# plt.ylim(0.3, 1.0)
# plt.ylim(0,1.2)
# plt.xlim(-1, len(c)+5)
# plt.savefig('figures/TUNING_ADAM_FEMNIST_ALL.pdf')
# plt.savefig('figures/TUNING_ADAM_FEMNIST.pdf')
plt.title(f"Tuning FedAdam on {d.upper()} with RS ({N} configs)")
plt.ylabel("Error Rate")
# plt.xscale('log')
plt.show()

In [ ]:
d = "cifar10"
# ed = eval_dicts[d]
runs = np.array(list(set(run for run, _, _ in ed)))
slrs = np.array([float(run.split('-betas')[0].split('lr_')[1]) for run in runs])
## Think about the rate at which good/bad configs are added (growing sides of the window)
sub_eds = []
rs_lr_nc_trials = defaultdict(lambda: defaultdict(lambda: []))
# center = -2
center = np.log(0.0014) / np.log(10)
# center = np.log(0.0005) / np.log(10)
I = np.arange(1,11,3)
for i in I:
    prev = 10**(center-0.1*i)
    curr = 10**(center+0.1*i)
    print(curr)
    lr_runs = runs[(slrs < curr) & (slrs > prev)]
    sub_ed = {
        k:v for k,v in ed.items() if k[0] in lr_runs
    }
    NC = get_NC(d)
    for nc in NC:
        for trial in range(200):
            rs_lr_nc_trials[i][nc].append(run_rs(sizes[d], lr_runs, sub_ed, nc, 0, NC[-1], N=6, R=405))

0.001762495576511837
0.0035166410041134162
0.007016621270781825
0.01400000000000002
